In [1]:
import pandas as pd
from nltk.corpus import stopwords
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
users_df = pd.read_excel("../data/Visitors Preference Dataset.xlsx")
places_df = pd.read_excel("../data/Places Dataset.xlsx")

In [3]:
users_df.head()

,User ID,Name,Email,Preferred Activities,Bucket list destinations Sri Lanka
0,1,Jennifer Quinn,jennifer.quinn@example.com,"['cycling', 'historical monuments', 'village h...","['Polonnaruwa', 'Hatton', 'Anuradhapura', 'Ell..."
1,2,Emily Perry,emily.perry@example.com,"['butterfly watching', 'hot springs', 'wildlif...","['Madunagala Hot Water Spring', 'Wilpattu Nati..."
2,3,Danielle Mcbride,danielle.mcbride@example.com,"['sea cruises', 'themed parks', 'craft worksho...","['Mirissa Beach', 'Negombo Lagoon', 'Batadomba..."
3,4,Angelica Wilson,angelica.wilson@example.com,"['fishing', 'hot springs', 'sailing']","['Maha Oya Hot Water Springs', 'Colombo Port C..."
4,5,Laurie Powers,laurie.powers@example.com,"['history tours', 'sailing', 'literary tours']","['Negombo Lagoon', 'Colombo Port City', 'Galle..."


In [4]:
places_df.head()

,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews
0,Arugam Bay Beach,6.840408,81.836848,"Arugam Bay Beach, Sri Lanka",4.8,1591.0,['Arugam Bay Beach is a surfer's paradise! I s...
1,Mirissa Beach,5.944703,80.459161,"Mirissa, Sri Lanka",4.6,1748.0,['Mirissa Beach is truly a gem on Sri LankaÃ¢Â...
2,Weligama Beach (surf and stay),5.972486,80.435714,"Weligama, Sri Lanka",4.4,325.0,['Weligama Beach is a fantastic spot for both ...
3,Ahangama,5.973975,80.362159,"Ahangama, Sri Lanka",NaN,NaN,['Ahangama was a bit disappointing for me as a...
4,Hikkaduwa Beach,6.137727,80.099060,"Hikkaduwa Beach, Sri Lanka",4.7,1438.0,['Hikkaduwa Beach is a delightful escape for s...


In [5]:
selected_features_user = ['Preferred Activities', 'Bucket list destinations Sri Lanka']
selected_features_places = ['name', 'latest_reviews']

In [6]:
users_df = users_df[selected_features_user]
places_df = places_df[selected_features_places]

In [7]:
users_df.fillna("", inplace=True)
places_df.fillna("", inplace=True)

In [8]:
users_df['Preferred Activities'] = users_df['Preferred Activities'].apply(lambda x: ' '.join(x.split()))
users_df['Bucket list destinations Sri Lanka'] = users_df['Bucket list destinations Sri Lanka'].apply(lambda x: ' '.join(x.split()))

In [9]:
users_df['Preferred Activities'] = users_df['Preferred Activities'].apply(
    lambda x: eval(x) if isinstance(x, str) and x.strip() != '' else ''
)
users_df['Bucket list destinations Sri Lanka'] = users_df['Bucket list destinations Sri Lanka'].apply(
    lambda x: eval(x) if isinstance(x, str) and x.strip() != '' else ''
)

In [10]:
users_df['Preferred Activities'] = users_df['Preferred Activities'].apply(lambda x: " ".join(x))
users_df['Bucket list destinations Sri Lanka'] = users_df['Bucket list destinations Sri Lanka'].apply(lambda x: " ".join(x))

In [11]:
users_df.head()

,Preferred Activities,Bucket list destinations Sri Lanka
0,cycling historical monuments village homestays,Polonnaruwa Hatton Anuradhapura Ella Haputale
1,butterfly watching hot springs wildlife viewing,Madunagala Hot Water Spring Wilpattu National ...
2,sea cruises themed parks craft workshops,Mirissa Beach Negombo Lagoon Batadombalena Cra...
3,fishing hot springs sailing,Maha Oya Hot Water Springs Colombo Port City N...
4,history tours sailing literary tours,Negombo Lagoon Colombo Port City Galle Dutch F...


In [12]:
places_df['latest_reviews'] = places_df['latest_reviews'].apply(
    lambda x: ''.join([i if i.isalpha() or i.isspace() else '' for i in x])
)

In [13]:
places_df['latest_reviews'] = places_df['latest_reviews'].apply(lambda x: x.lower())
places_df['name'] = places_df['name'].apply(lambda x: x.lower())
users_df['Preferred Activities'] = users_df['Preferred Activities'].apply(lambda x: x.lower())
users_df['Bucket list destinations Sri Lanka'] = users_df['Bucket list destinations Sri Lanka'].apply(lambda x: x.lower())

In [14]:
stop_words = set(stopwords.words('english'))

places_df['latest_reviews'] = places_df['latest_reviews'].apply(
    lambda x: ' '.join([word for word in x.split() if word not in stop_words])
)

In [15]:
places_df.head()

,name,latest_reviews
0,arugam bay beach,arugam bay beach surfers paradise spent incred...
1,mirissa beach,mirissa beach truly gem sri lankaãââs southern...
2,weligama beach (surf and stay),weligama beach fantastic spot beginner experie...
3,ahangama,ahangama bit disappointing solo traveler surfi...
4,hikkaduwa beach,hikkaduwa beach delightful escape solo travele...


In [16]:
combined_features_places = places_df['name'] + ' ' + places_df['latest_reviews']
combined_features_users = users_df['Preferred Activities'] + ' ' + users_df['Bucket list destinations Sri Lanka']

In [17]:
combined_features_places[0], combined_features_users[0]

('arugam bay beach arugam bay beach surfers paradise spent incredible days riding waves local surf schools fantastic beginners like atmosphere laidback friendly locals fellow travelers long day surfing sunsets simply magical beach bit crowded especially peak season adds lively vibe canãâât wait return friends unforgettable time arugam bay beach surfing conditions excellent managed catch great waves beach beautiful soft sand clear waters perfect swimming however noticed litter beach bit disappointing overall vibrant nightlife delicious food made definitely worth visit couple looking relaxation arugam bay beach offered perfect blend tranquility excitement enjoyed lazy days lounging beach indulging fresh seafood beachside restaurants surf scene lively easy find quieter spots unwind downside occasional noise nearby parties didnãâât detract much experience lovely getaway visited arugam bay beach family children loved surf lessons found beach bit overcrowded atmosphere vibrant locals warm we

In [18]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix_users = tfidf_vectorizer.fit_transform(combined_features_users)
tfidf_matrix_places = tfidf_vectorizer.transform(combined_features_places)

In [19]:
tfidf_matrix_places.shape

(411, 293)

In [20]:
tfidf_matrix_users.shape

(10000, 293)

In [21]:
cosine_sim = cosine_similarity(tfidf_matrix_users, tfidf_matrix_places)

In [22]:
cosine_sim.shape

(10000, 411)

In [23]:
cosine_sim[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.06155764, 0.25674769,
       0.10321083, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.31206528, 0.04395955,
       0.        , 0.09391946, 0.06347571, 0.        , 0.08208183,
       0.07879243, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.05971237, 0.        ,
       0.        , 0.        , 0.05959577, 0.        , 0.0626752 ,
       0.        , 0.        , 0.        , 0.05208398, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [24]:
top_5_places = cosine_sim[0].argsort()[-5:][::-1]

In [25]:
places_original = pd.read_excel("../data/Places Dataset.xlsx")
users_original = pd.read_excel("../data/Visitors Preference Dataset.xlsx")

In [26]:
places_original.iloc[top_5_places]

,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews
95,Polonnaruwa,7.940338,81.018798,"Polonnaruwa, Sri Lanka",NaN,NaN,['Polonnaruwa felt like a missed opportunity. ...
23,Anuradhapura,8.311352,80.403651,"Anuradhapura, Sri Lanka",NaN,NaN,"['Anuradhapura is rich in history, but I found..."
272,Lotus Pond (Nelum pokuna II (ÃƒÂ Ã‚Â¶Ã‚Â±ÃƒÂ Ã...,7.983824,81.003876,"Polonnaruwa, Sri Lanka",3.8,87.0,['Lotus Pond is a hidden gem in Polonnaruwa! T...
9,Polonnaruwa Ancient City,7.945942,81.000329,"Polonnaruwa, Sri Lanka",4.3,878.0,['Polonnaruwa Ancient City is a stunning place...
78,Haputale,6.765414,80.952565,"Haputale, Sri Lanka",NaN,NaN,['Haputale is a beautiful location with stunni...


In [27]:
for i in top_5_places:
    print(f"Similarity measure for place {places_original.iloc[i]['name']} is {cosine_sim[0][i]}")

Similarity measure for place Polonnaruwa is 0.321558044441415
Similarity measure for place Anuradhapura is 0.31206528276255785
Similarity measure for place Lotus Pond (Nelum pokuna II (ÃƒÂ Ã‚Â¶Ã‚Â±ÃƒÂ Ã‚Â·Ã‚Â™ÃƒÂ Ã‚Â·Ã‚Â…ÃƒÂ Ã‚Â·Ã‚Â”ÃƒÂ Ã‚Â¶Ã‚Â¸ÃƒÂ Ã‚Â·Ã‚ÂŠ ÃƒÂ Ã‚Â¶Ã‚Â´ÃƒÂ Ã‚Â·Ã‚ÂœÃƒÂ Ã‚Â¶Ã‚ÂšÃƒÂ Ã‚Â·Ã‚Â”ÃƒÂ Ã‚Â¶Ã‚Â« II)) is 0.25868718538124624
Similarity measure for place Polonnaruwa Ancient City is 0.25674769048579826
Similarity measure for place Haputale is 0.24510574723355055


In [28]:
users_original.iloc[0]

User ID                                                                               1
Name                                                                     Jennifer Quinn
Email                                                        jennifer.quinn@example.com
Preferred Activities                  ['cycling', 'historical monuments', 'village h...
Bucket list destinations Sri Lanka    ['Polonnaruwa', 'Hatton', 'Anuradhapura', 'Ell...
Name: 0, dtype: object

In [29]:
# Let's put everything in a function
def get_recommendations(user_id):
    user = users_original.iloc[user_id]
    user_features = user['Preferred Activities'] + ' ' + user['Bucket list destinations Sri Lanka']
    user_features = user_features.lower()
    print(f"User features: {user_features}")
    user_features = ' '.join([word for word in user_features.split() if word not in stop_words])
    user_features = [user_features]
    user_tfidf = tfidf_vectorizer.transform(user_features)
    cosine_sim = cosine_similarity(user_tfidf, tfidf_matrix_places)
    top_5_places = cosine_sim[0].argsort()[-10:][::-1]
    for i in top_5_places:
        print(f"Similarity measure for place {places_original.iloc[i]['name']} is {cosine_sim[0][i]}")
    return places_original.iloc[top_5_places]

In [30]:
get_recommendations(0)

User features: ['cycling', 'historical monuments', 'village homestays'] ['polonnaruwa', 'hatton', 'anuradhapura', 'ella', 'haputale']
Similarity measure for place Polonnaruwa is 0.32155804444141506
Similarity measure for place Anuradhapura is 0.31206528276255796
Similarity measure for place Lotus Pond (Nelum pokuna II (ÃƒÂ Ã‚Â¶Ã‚Â±ÃƒÂ Ã‚Â·Ã‚Â™ÃƒÂ Ã‚Â·Ã‚Â…ÃƒÂ Ã‚Â·Ã‚Â”ÃƒÂ Ã‚Â¶Ã‚Â¸ÃƒÂ Ã‚Â·Ã‚ÂŠ ÃƒÂ Ã‚Â¶Ã‚Â´ÃƒÂ Ã‚Â·Ã‚ÂœÃƒÂ Ã‚Â¶Ã‚ÂšÃƒÂ Ã‚Â·Ã‚Â”ÃƒÂ Ã‚Â¶Ã‚Â« II)) is 0.2586871853812463
Similarity measure for place Polonnaruwa Ancient City is 0.2567476904857983
Similarity measure for place Haputale is 0.24510574723355058
Similarity measure for place Alahana Pirivena is 0.224432706088184
Similarity measure for place Potgul Temple is 0.2241105035938603
Similarity measure for place Hatadage is 0.21298525380614178
Similarity measure for place Anuradhapura New Town is 0.21182306005735435
Similarity measure for place Polonnaruwa Sivan Kovil is 0.2034052339161116


,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews
95,Polonnaruwa,7.940338,81.018798,"Polonnaruwa, Sri Lanka",NaN,NaN,['Polonnaruwa felt like a missed opportunity. ...
23,Anuradhapura,8.311352,80.403651,"Anuradhapura, Sri Lanka",NaN,NaN,"['Anuradhapura is rich in history, but I found..."
272,Lotus Pond (Nelum pokuna II (ÃƒÂ Ã‚Â¶Ã‚Â±ÃƒÂ Ã...,7.983824,81.003876,"Polonnaruwa, Sri Lanka",3.8,87.0,['Lotus Pond is a hidden gem in Polonnaruwa! T...
9,Polonnaruwa Ancient City,7.945942,81.000329,"Polonnaruwa, Sri Lanka",4.3,878.0,['Polonnaruwa Ancient City is a stunning place...
78,Haputale,6.765414,80.952565,"Haputale, Sri Lanka",NaN,NaN,['Haputale is a beautiful location with stunni...
366,Alahana Pirivena,7.961924,81.003995,"Polonnaruwa, Sri Lanka",4.8,121.0,"[""Visiting Alahana Pirivena was a spiritual jo..."
284,Potgul Temple,7.924544,80.994533,"Polonnaruwa, Sri Lanka",4.4,553.0,['Visiting Potgul Temple was a captivating jou...
276,Hatadage,7.947864,81.001302,"Polonnaruwa, Sri Lanka",4.5,375.0,['Visiting Hatadage was a truly enriching expe...
315,Anuradhapura New Town,8.322765,80.402577,"Anuradhapura, Sri Lanka",4.4,167.0,['Anuradhapura New Town is a fascinating blend...
368,Polonnaruwa Sivan Kovil,7.946288,81.001328,"Polonnaruwa, Sri Lanka",4.8,146.0,['Visiting Polonnaruwa Sivan Kovil was a fasci...


In [31]:
get_recommendations(1)

User features: ['butterfly watching', 'hot springs', 'wildlife viewing'] ['madunagala hot water spring', 'wilpattu national park', 'wasgamuwa national park', 'kanneliya national rain forest reserve', 'horton plains national park']
Similarity measure for place Wilpattu National Park is 0.5631532210283656
Similarity measure for place Madunagala Hot Water Spring is 0.5369587925101469
Similarity measure for place Wasgamuwa National Park is 0.5369477319501574
Similarity measure for place Somawathiya National Park is 0.5228311978255588
Similarity measure for place Ussangoda National Park | ÃƒÂ Ã‚Â¶Ã‚Â‹ÃƒÂ Ã‚Â·Ã‚ÂƒÃƒÂ Ã‚Â·Ã‚ÂŠÃƒÂ Ã‚Â·Ã‚ÂƒÃƒÂ Ã‚Â¶Ã‚Â±ÃƒÂ Ã‚Â·Ã‚ÂŠÃƒÂ Ã‚Â¶Ã‚ÂœÃƒÂ Ã‚Â·Ã‚ÂœÃƒÂ Ã‚Â¶Ã‚Â© ÃƒÂ Ã‚Â¶Ã‚Â¢ÃƒÂ Ã‚Â·Ã‚Â_x008f_ÃƒÂ Ã‚Â¶Ã‚Â­ÃƒÂ Ã‚Â·Ã‚Â’ÃƒÂ Ã‚Â¶Ã‚Âš ÃƒÂ Ã‚Â¶Ã‚Â‹ÃƒÂ Ã‚Â¶Ã‚Â¯ÃƒÂ Ã‚Â·Ã‚ÂŠÃƒÂ¢Ã‚Â€Ã‚Â_x008d_ÃƒÂ Ã‚Â¶Ã‚ÂºÃƒÂ Ã‚Â·Ã‚Â_x008f_ÃƒÂ Ã‚Â¶Ã‚Â±ÃƒÂ Ã‚Â¶Ã‚Âº is 0.5019767144883026
Similarity measure for place Kanneliya National Rain Forest Reserve | ÃƒÂ Ã‚Â¶Ã‚ÂšÃƒÂ Ã‚Â¶Ã‚Â±ÃƒÂ Ã‚Â·

,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews
13,Wilpattu National Park,8.458182,80.051831,Sri Lanka,4.7,2708.0,['Wilpattu National Park is a fantastic destin...
124,Madunagala Hot Water Spring,6.253630,80.981711,Sri Lanka,4.6,1117.0,['Madunagala Hot Water Spring is a hidden gem!...
14,Wasgamuwa National Park,7.753242,80.920774,"Nuwaragala, Sri Lanka",4.3,963.0,['Wasgamuwa National Park exceeded my expectat...
271,Somawathiya National Park,8.169256,81.207816,"Gangapahalavila, Sri Lanka",4.3,719.0,['Somawathiya National Park is a hidden gem in...
203,Ussangoda National Park | ÃƒÂ Ã‚Â¶Ã‚Â‹ÃƒÂ Ã‚Â·...,6.098920,80.988343,"Ussangoda, Sri Lanka",4.7,674.0,['Ussangoda National Park was an unexpected ge...
403,Kanneliya National Rain Forest Reserve | ÃƒÂ Ã...,6.257642,80.359621,"Sri, Sri Lanka",4.7,143.0,['Kanneliya National Rain Forest Reserve is a ...
269,Angammedilla National Park,7.925445,80.920774,"Diyabeduma, Sri Lanka",4.1,287.0,['Angammedilla National Park is a hidden gem! ...
399,Madunagala Hot Water Spring,6.253630,80.981711,Sri Lanka,4.5,1151.0,['Madunagala Hot Water Spring is a hidden gem!...
15,Minneriya National Park,8.015644,80.844670,Sri Lanka,4.6,1293.0,['Minneriya National Park is a must-visit for ...
17,Horton Plains National Park,6.809446,80.802333,Sri Lanka,4.7,8564.0,"[\n""Horton Plains National Park was an incredi..."


In [32]:
get_recommendations(2)

User features: ['sea cruises', 'themed parks', 'craft workshops'] ['mirissa beach', 'negombo lagoon', 'batadombalena craft centre', 'jungle beach', 'bentota']
Similarity measure for place Jungle Beach is 0.3886152826261924
Similarity measure for place Cinnamon Bentota Beach is 0.3229426131361829
Similarity measure for place Mullaitivu Beach is 0.3160128484790499
Similarity measure for place ÃƒÂ Ã‚Â¶Ã‚ÂšÃƒÂ Ã‚Â·Ã‚Â_x009d_ÃƒÂ Ã‚Â¶Ã‚ÂšÃƒÂ Ã‚Â·Ã‚Â’ÃƒÂ Ã‚Â·Ã‚Â…ÃƒÂ Ã‚Â·Ã‚Â_x008f_ÃƒÂ Ã‚Â¶Ã‚ÂºÃƒÂ Ã‚Â·Ã‚Â’ ÃƒÂ Ã‚Â·Ã‚Â€ÃƒÂ Ã‚Â·Ã‚Â™ÃƒÂ Ã‚Â¶Ã‚Â»ÃƒÂ Ã‚Â·Ã‚Â… |Kokkilai Beach is 0.3159314039754804
Similarity measure for place Mirissa is 0.3100937172787711
Similarity measure for place Okanda Beach is 0.30807277369679764
Similarity measure for place Mirissa Beach is 0.2891311757296992
Similarity measure for place SUNDARAM BEACH is 0.2847907431564123
Similarity measure for place Thambiluvil Beach is 0.28455347224300714
Similarity measure for place Rumassala South Beach is 0.28277748981714806


,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews
33,Jungle Beach,6.018694,80.239410,"Jungle Beach, Sri Lanka",4.6,2395.0,['Jungle Beach is a hidden gem! The trek to ge...
66,Cinnamon Bentota Beach,6.424905,79.996875,"Bentota , Sri Lanka",4.9,3929.0,['Cinnamon Bentota Beach was a delightful esca...
259,Mullaitivu Beach,9.271618,80.819464,"Mullaitivu, Sri Lanka",4.1,708.0,['Mullaitivu Beach is a hidden gem! The soft s...
365,ÃƒÂ Ã‚Â¶Ã‚ÂšÃƒÂ Ã‚Â·Ã‚Â_x009d_ÃƒÂ Ã‚Â¶Ã‚ÂšÃƒÂ ...,9.064594,80.918706,Sri Lanka,3.9,140.0,['Kokkilai Beach is a hidden gem! The tranquil...
46,Mirissa,5.948262,80.471587,"Mirissa, Sri Lanka",NaN,NaN,"['I recently went on a sea cruise in Mirissa, ..."
255,Okanda Beach,6.650140,81.776187,"Okanda, Sri Lanka",4.0,154.0,['Okanda Beach is a hidden gem on the Sri Lank...
1,Mirissa Beach,5.944703,80.459161,"Mirissa, Sri Lanka",4.6,1748.0,['Mirissa Beach is truly a gem on Sri LankaÃ¢Â...
339,SUNDARAM BEACH,9.622718,80.005239,"Jaffna, Sri Lanka",4.8,76.0,['Sundaram Beach is an absolute gem! The prist...
141,Thambiluvil Beach,7.125067,81.857297,"Thambiluvil, Sri Lanka",4.8,88.0,['Thambiluvil Beach is an absolute gem! The pr...
182,Rumassala South Beach,6.018463,80.235487,"Unawatuna, Sri Lanka",5.0,267.0,['Rumassala South Beach is a serene spot that ...


In [33]:
get_recommendations(3)

User features: ['fishing', 'hot springs', 'sailing'] ['maha oya hot water springs', 'colombo port city', 'negombo lagoon', 'trincomalee harbour', 'kalpitiya']
Similarity measure for place Maha Oya Hot Water Springs is 0.7310946615306441
Similarity measure for place Madunagala Hot Water Spring is 0.5216032989861011
Similarity measure for place Kanniya Hot Water Springs is 0.5195136492959672
Similarity measure for place Madunagala Hot Water Spring is 0.4662498688208775
Similarity measure for place Port City Colombo is 0.3168452244428236
Similarity measure for place Kalpitiya Lagoon is 0.3134944464099639
Similarity measure for place Jayanthi Wewa Hot Spring is 0.29055218543131034
Similarity measure for place Ruwanweli Maha Seya is 0.2876554196123194
Similarity measure for place Nanu Oya Water Falls, Nanu Oya is 0.2859008515128844
Similarity measure for place Negombo Lagoon is 0.2749945788140384


,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews
123,Maha Oya Hot Water Springs,7.552181,81.353041,"Maha Oya Hot Water Springs, Sri Lanka",4.3,180.0,['Maha Oya Hot Water Springs is a hidden gem! ...
399,Madunagala Hot Water Spring,6.253630,80.981711,Sri Lanka,4.5,1151.0,['Madunagala Hot Water Spring is a hidden gem!...
122,Kanniya Hot Water Springs,8.604498,81.171338,"Trincomalee, Sri Lanka",4.3,2538.0,['Kanniya Hot Water Springs is a delightful sp...
124,Madunagala Hot Water Spring,6.253630,80.981711,Sri Lanka,4.6,1117.0,['Madunagala Hot Water Spring is a hidden gem!...
47,Port City Colombo,6.937804,79.836843,"Port City Colombo, Sri Lanka",NaN,NaN,"['Port City Colombo has a lot of potential, bu..."
74,Kalpitiya Lagoon,8.241272,79.744194,"Kalpitiya Lagoon, Sri Lanka",4.8,130.0,['Kalpitiya Lagoon is a fantastic destination ...
312,Jayanthi Wewa Hot Spring,7.140848,81.552397,"Wadinagala, Sri Lanka",4.2,56.0,['Jayanthi Wewa Hot Spring was a delightful su...
146,Ruwanweli Maha Seya,8.349893,80.396556,"Anuradhapura, Sri Lanka",4.6,15142.0,['Visiting Ruwanweli Maha Seya was a highlight...
264,"Nanu Oya Water Falls, Nanu Oya",6.945153,80.738782,"Nanuoya, Sri Lanka",4.8,507.0,['Nanu Oya Water Falls is an absolute gem! The...
45,Negombo Lagoon,7.158004,79.846480,"Negombo Lagoon, Sri Lanka",4.5,268.0,['Negombo Lagoon was a delightful escape! We t...


In [34]:
get_recommendations(4)

User features: ['history tours', 'sailing', 'literary tours'] ['negombo lagoon', 'colombo port city', 'galle dutch fort', 'sigiriya', 'jaffna public library']
Similarity measure for place Jaffna Public Library is 0.5285583991947298
Similarity measure for place Dutch Fort - Jaffna is 0.47331434369213193
Similarity measure for place Galle Dutch Fort is 0.45502195415155994
Similarity measure for place Black Galle Fort - Zwart Bastion is 0.3719584909147525
Similarity measure for place Fort Entrance (Old Gate) is 0.3600323025718844
Similarity measure for place Batticaloa Dutch Fort is 0.3545358109558287
Similarity measure for place Dutch Reformed Church, Matara is 0.35427947989615083
Similarity measure for place Our Lady of Refuge (OLR) Church, Jaffna | ÃƒÂ Ã‚Â®Ã‚Â…ÃƒÂ Ã‚Â®Ã‚ÂŸÃƒÂ Ã‚Â¯Ã‚ÂˆÃƒÂ Ã‚Â®Ã‚Â•ÃƒÂ Ã‚Â¯Ã‚Â_x008d_ÃƒÂ Ã‚Â®Ã‚Â•ÃƒÂ Ã‚Â®Ã‚Â² ÃƒÂ Ã‚Â®Ã‚Â®ÃƒÂ Ã‚Â®Ã‚Â¾ÃƒÂ Ã‚Â®Ã‚Â¤ÃƒÂ Ã‚Â®Ã‚Â¾ ÃƒÂ Ã‚Â®Ã‚Â•ÃƒÂ Ã‚Â¯Ã‚Â‹ÃƒÂ Ã‚Â®Ã‚ÂµÃƒÂ Ã‚Â®Ã‚Â¿ÃƒÂ Ã‚Â®Ã‚Â²ÃƒÂ Ã‚Â¯Ã‚Â_x008d_ | ÃƒÂ Ã‚Â¶Ã‚Â…ÃƒÂ Ã‚Â¶

,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews
53,Jaffna Public Library,9.662128,80.011731,"Jaffna, Sri Lanka",4.9,575.0,['The Jaffna Public Library is a beautiful hom...
211,Dutch Fort - Jaffna,9.662031,80.008425,"Jaffna, Sri Lanka",4.7,3672.0,['The Dutch Fort in Jaffna is a splendid testa...
8,Galle Dutch Fort,6.030459,80.215021,"Galle 80000, Sri Lanka",4.6,16934.0,"[""Galle Dutch Fort is a stunning blend of hist..."
194,Black Galle Fort - Zwart Bastion,6.029129,80.219292,"Galle, Sri Lanka",4.3,243.0,"['The Black Galle Fort, or Zwart Bastion, is a..."
195,Fort Entrance (Old Gate),6.028114,80.218447,ÃƒÂ Ã‚Â¶Ã‚Â¸ÃƒÂ Ã‚Â·Ã‚Â_x008f_ÃƒÂ Ã‚Â¶Ã‚Â­ÃƒÂ ...,4.8,231.0,"['The Fort Entrance, or Old Gate, is a captiva..."
165,Batticaloa Dutch Fort,7.711813,81.702124,"Batticaloa, Sri Lanka",4.0,1189.0,['The Batticaloa Dutch Fort is a hidden gem! I...
178,"Dutch Reformed Church, Matara",5.944788,80.546927,"Matara, Sri Lanka",4.7,37.0,['The Dutch Reformed Church in Matara is a hid...
336,"Our Lady of Refuge (OLR) Church, Jaffna | ÃƒÂ ...",9.663146,80.018255,"Jaffna, Sri Lanka",4.3,160.0,['Visiting Our Lady of Refuge Church in Jaffna...
192,Galle Fort Attractions and Jumpers Sri Lanka,6.030464,80.215024,"Galle, Sri Lanka",4.7,1328.0,['Galle Fort is an enchanting blend of history...
242,Mannar Dutch Fort,8.975856,79.916996,"Mannar, Sri Lanka",4.4,817.0,['Visiting Mannar Dutch Fort was a delightful ...


In [35]:
get_recommendations(5)

User features: ['public art installations', 'temple pilgrimages', 'architecture tours'] ['colombo', 'sigiriya', 'mihintale', 'galle dutch fort', 'dambulla royal cave temple and golden temple']
Similarity measure for place Dambulla Royal Cave Temple and Golden Temple is 0.6016178877728274
Similarity measure for place Dambulla Royal Cave Temple and Golden Temple is 0.574573689545917
Similarity measure for place Manelwatta Temple is 0.569871433691222
Similarity measure for place Korathota Royal Temple is 0.5345759993538072
Similarity measure for place ÃƒÂ Ã‚Â¶Ã‚Â´ÃƒÂ Ã‚Â¶Ã‚Â»ÃƒÂ Ã‚Â·Ã‚Â™ÃƒÂ Ã‚Â·Ã‚Â€ÃƒÂ Ã‚Â·Ã‚Â’ ÃƒÂ Ã‚Â¶Ã‚Â¯ÃƒÂ Ã‚Â·Ã‚Â–ÃƒÂ Ã‚Â·Ã‚Â€ ÃƒÂ Ã‚Â·Ã‚Â€ÃƒÂ Ã‚Â·Ã‚Â’ÃƒÂ Ã‚Â·Ã‚Â„ÃƒÂ Ã‚Â·Ã‚Â_x008f_ÃƒÂ Ã‚Â¶Ã‚Â»ÃƒÂ Ã‚Â·Ã‚ÂƒÃƒÂ Ã‚Â·Ã‚ÂŠÃƒÂ Ã‚Â¶Ã‚Â®ÃƒÂ Ã‚Â·Ã‚Â_x008f_ÃƒÂ Ã‚Â¶Ã‚Â±ÃƒÂ Ã‚Â¶Ã‚Âº - Parewi Duwa Temple is 0.5070119432729592
Similarity measure for place Sambodhi Pagoda Temple is 0.5009065298645088
Similarity measure for place Single Tree Hill is 0.47815438529529614
Similarity measu

,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews
395,Dambulla Royal Cave Temple and Golden Temple,7.854914,80.650570,"Dambulla, Sri Lanka",4.3,11690.0,['Dambulla Royal Cave Temple is a must-visit f...
22,Dambulla Royal Cave Temple and Golden Temple,7.854914,80.650570,"Dambulla, Sri Lanka",4.2,11598.0,['The Dambulla Royal Cave Temple is a breathta...
342,Manelwatta Temple,6.944921,79.951361,Sri Lanka,4.8,1673.0,['Visiting Manelwatta Temple was a serene expe...
330,Korathota Royal Temple,6.915038,80.002135,Sri Lanka,4.8,1226.0,['Korathota Royal Temple is a hidden gem! The ...
187,ÃƒÂ Ã‚Â¶Ã‚Â´ÃƒÂ Ã‚Â¶Ã‚Â»ÃƒÂ Ã‚Â·Ã‚Â™ÃƒÂ Ã‚Â·Ã‚...,5.941381,80.549255,"Matara, Sri Lanka",4.4,947.0,['Visiting Paravi Duwa Temple was a delightful...
329,Sambodhi Pagoda Temple,6.938393,79.842028,"Colombo , Sri Lanka",4.8,394.0,['Sambodhi Pagoda Temple is a hidden gem! The ...
266,Single Tree Hill,6.957785,80.762501,"Nuwara Eliya, Sri Lanka",4.3,629.0,['Single Tree Hill offers breathtaking panoram...
183,ParappudÃƒÂ…Ã‚Â«va Island Temple,6.111568,80.138342,"Rathgama, Sri Lanka",4.5,136.0,['ParappudÃƒÂ¥va Island Temple is a hidden gem...
181,SrÃƒÂ„Ã‚Â« VijayÃƒÂ„Ã‚Â_x0081_rÃƒÂ„Ã‚Â_x0081_m...,6.245269,80.057509,"Ambalangoda, Sri Lanka",4.8,239.0,['Visiting SrÃƒÂ¯ VijayÃ„Â_x0081_rÃ„Â_x0081_ma...
267,Swarnagiri Maha Viharaya / Single Tree Temple,6.955056,80.766179,"Nuwara Eliya, Sri Lanka",4.5,372.0,['Swarnagiri Maha Viharaya is a hidden gem! Th...


In [36]:
get_recommendations(6)

User features: ['fishing', 'golfing', 'historical monuments'] ['hikkaduwa', 'kalpitiya', 'polonnaruwa', 'negombo lagoon', 'nuwara eliya golf club']
Similarity measure for place Nuwara Eliya Golf Club is 0.5623215404784443
Similarity measure for place Royal Colombo Golf Club is 0.41607514437174287
Similarity measure for place Kalpitiya Lagoon is 0.3677508569819012
Similarity measure for place Nuwara Eliya Mountains view point is 0.31178162578745117
Similarity measure for place Polonnaruwa is 0.2877315701719622
Similarity measure for place Panakala Lagoon is 0.2839185622853645
Similarity measure for place Nuwara Eliya is 0.2779983999278214
Similarity measure for place Kalpitiya is 0.26814452544817974
Similarity measure for place Batticaloa Lagoon is 0.2656503137878814
Similarity measure for place Negombo Lagoon is 0.24468269018960098


,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews
64,Nuwara Eliya Golf Club,6.971737,80.765731,"Nuwara Eliya , Sri Lanka",4.1,1129.0,['Nuwara Eliya Golf Club offers a unique golfi...
65,Royal Colombo Golf Club,6.905227,79.884200,"Colombo, Sri Lanka",4.4,1945.0,['The Royal Colombo Golf Club is a true oasis ...
74,Kalpitiya Lagoon,8.241272,79.744194,"Kalpitiya Lagoon, Sri Lanka",4.8,130.0,['Kalpitiya Lagoon is a fantastic destination ...
222,Nuwara Eliya Mountains view point,6.983455,80.750476,"Nuwara Eliya, Sri Lanka",4.0,163.0,['The Nuwara Eliya Mountains viewpoint is trul...
95,Polonnaruwa,7.940338,81.018798,"Polonnaruwa, Sri Lanka",NaN,NaN,['Polonnaruwa felt like a missed opportunity. ...
256,Panakala Lagoon,6.732608,81.793739,"Lahugala, Sri Lanka",NaN,NaN,['Panakala Lagoon was a bit of a letdown. The ...
77,Nuwara Eliya,6.949717,80.789107,"Nuwara Eliya, Sri Lanka",NaN,NaN,['Nuwara Eliya is a bit overrated for its char...
59,Kalpitiya,8.229528,79.759614,"Kalpitiya, Sri Lanka",NaN,NaN,"['Kalpitiya has beautiful scenery, but I was d..."
326,Batticaloa Lagoon,7.714400,81.702570,"Batticaloa, Sri Lanka",3.8,121.0,['Batticaloa Lagoon is a hidden gem! The seren...
45,Negombo Lagoon,7.158004,79.846480,"Negombo Lagoon, Sri Lanka",4.5,268.0,['Negombo Lagoon was a delightful escape! We t...
